# Notes

result_id 10040 and 10041 have None for Service Type ( change to CAF?)

result id 10002 has a null Benefit (set to Disability Pension?)

In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 5000

# Get data

In [3]:
df_mapping_orig = pd.read_excel('../data/VAC Data.xlsx', sheet_name=2)
df_results_orig = pd.read_excel('../data/VAC Data.xlsx', sheet_name=3)

# Clean and munge data

### Mapping data

In [4]:
df_mapping = df_mapping_orig.rename(columns={df_mapping_orig.columns[0]: "Result ID", 
                                         "Statements": "Scenario",
                                         "Unnamed: 129": "Service Type",
                                         "Unnamed: 127": "Result Name",
                                         "Unnamed: 130": "Applicant",
                                         "Unnamed: 131": "Benefit",
                                         "Unnamed: 132": "Additional Details",
                                         "Unnamed: 128": "Result Description",
                                        })
df_mapping = df_mapping.drop(df_mapping.index[0])

In [5]:
#df_mapping[df_mapping['Service Type'].isnull()]
df_mapping['Service Type'] = df_mapping['Service Type'].fillna('CAF')

In [6]:
#df_mapping[df_mapping['Benefit'].isnull()]
df_mapping.loc[df_mapping['Result ID'] == 10002, 'Benefit'] = 'Disability Pension'

### Result ID data

In [7]:
df_results = df_results_orig[:]

df_results.head()
df_results[['ID', 'Title (English)', 'Content (English)']]
df_results['Name'] = df_results['EN MVA'].map(lambda x: BeautifulSoup(x, "html5lib").find_all('h2')[0].get_text())
df_results['Description'] = df_results['EN MVA'].map(lambda xml:' '.join([y.get_text() for y in BeautifulSoup(xml, "html5lib").find_all('p')]))

df_results = df_results[['ID', 'Name', 'Title (English)', 'Description', 'Content (English)']].set_index('ID')

In [8]:
# The new Name and Description field are mostly the same as the existing Title and Content fields
df_results.head(1)

,Name,Title (English),Description,Content (English)
ID,,,,
1,Disability Benefits,Disability Benefits,"If there is medical evidence that shows that your condition has worsened, you may be eligible for an additional amount of compensation. To determine if you qualify, send a secure message or contact us to request a reassessment. However, if you have already received the maximum assessment, no additional amount can be provided. If you have a new condition related to your service, you can also apply for a disability benefit for this condition.","If there is medical evidence that shows that your condition has worsened, you may be eligible for an additional amount of compensation. To determine if you qualify, send a secure message or call us to request a reassessment. However, if you have already received the maximum assessment, no additional amount can be provided.\nIf you have a new condition related to your service, you can also apply for a Disability Benefit for this condition.\n"


In [9]:
# Except when the latter fields are empty
df_results.tail(1)

,Name,Title (English),Description,Content (English)
ID,,,,
10041,Veterans Independence Program (VIP) for primary caregivers,NaN,"Do you have questions about your VIP grant? If so, please contact us or consult the Frequently Asked Questions \nsection on VIP.",NaN


# Summarize by Result ID

Create a table with one row per result id. Have a column containing all `Applicant` and `Service Type` for that result.

In [10]:
df = df_mapping[['Result ID', 'Benefit', 'Applicant', 'Service Type']][:]
df['Applicant'] = df['Applicant'] + '; '
df['Service Type'] = df['Service Type'] + '; '
df['Benefit'] = df['Benefit'].astype(str)
df.head()

,Result ID,Benefit,Applicant,Service Type
11,1,Disability Benefits,Member;,CAF;
21,2,Disability Benefits,Member;,CAF;
31,3,Disability Benefits,Member;,War Service Veteran ;
32,3,Disability Benefits,Member;,RCMP;
33,3,Disability Benefits,Member;,Allied Veteran;


In [11]:
aggregation = {
    'Benefit': lambda x: x.unique().sum(),
    'Applicant': lambda x: x.unique().sum(),
    'Service Type':lambda x: x.unique().sum(),
}

df_mapping_summary = df.groupby(['Result ID']).agg(aggregation)
df_mapping_summary.rename(columns={'Applicant': 'Applicant Types', 'Service Type':'Service Types'}, inplace=True)
df_mapping_summary.reset_index(inplace=True)
df_mapping_summary = df_mapping_summary[['Benefit', 'Result ID', 'Applicant Types', 'Service Types']]

df_mapping_summary.set_index('Result ID', inplace=True)
df_mapping_summary = df_mapping_summary.join(df_results).reset_index()
df_mapping_summary = df_mapping_summary[['Benefit', 'Result ID', 'Description', 'Applicant Types', 'Service Types']]
df_mapping_summary.set_index(['Benefit', 'Result ID'], inplace=True)
#df_mapping_summary.to_csv('summary_result_id.csv')
df_mapping_summary

Description  \
Benefit                                              Result ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
Disability Benefits                                  1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            If there is medical evidence that shows that your condition has worsened, you may be eligible for an additional amount of compensation. To determine if you qualify, send a secure message or contact us to request a reassessment. However, if you have already received the maximum assessment, no additional amount can be provided. If you have a new condition related to your service, you can also apply for a disability benefit for this condition.   
                                                     2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       Disability benefits are financial payments made to you when you have a service-related disability. To receive a disability benefit you must have a diagnosed medical condition or disability and be able to show that the condition or disability is related to your service. The amount provided depends on the degree to which your condition is related to your service (entitlement) and the extent to which the disability impacts your health (assessment).   
                                                     3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            If there is medical evidence that shows that your condition has worsened, you may be eligible for an additional amount of compensation. To determine if you qualify, send a secure message or contact us to request a reassessment. However, if you have already received

# Summarize by Benefit

Create a table with one row per benefit. Have a column containing all `Applicant` and `Service Type` for that result.

In [12]:
df = df_mapping[['Benefit', 'Applicant', 'Service Type']][:]
df['Benefit'] = df['Benefit'].astype(str)
df['Applicant'] = df['Applicant'] + '; '
df['Service Type'] = df['Service Type'] + '; '

aggregation = {
    'Applicant': lambda x: x.unique().sum(),
    'Service Type':lambda x: x.unique().sum(),
}

df_mapping_summary = df.groupby(['Benefit']).agg(aggregation)
df_mapping_summary = df_mapping_summary.rename({'Applicant': 'Applicant Types', 'Service Type':'Service Types'})
df_mapping_summary

,Applicant,Service Type
Benefit,,
Attendance Allowance,Member; Spouse / Common-law Partner;,CAF; War Service Veteran ; RCMP; Allied Veteran;
Canadian Forces Income Supplement,Member; Survivor; Orphan (Veteran is deceased);,CAF;
Career Transition Services,Member;,CAF;
Case Management,Member;,CAF; RCMP; War Service Veteran ; Allied Veteran;
Clothing Allowance,Member;,CAF; War Service Veteran ; RCMP; Allied Veteran;
Critical Injury Benefit,Member;,CAF;
Death Benefit,Survivor; Orphan (Veteran is deceased);,CAF;
Disability Award,Survivor;,CAF;
Disability Benefits,Member; Spouse / Common-law Partner;,CAF; War Service Veteran ; RCMP; Allied Veteran;
